## RPFS Problem (TWCT objective) - Worstcase MIP Parametrization

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List files in the result folder 

In [109]:
resultfolder = os.path.join(os.getcwd(), 'results')
rpfs_file = os.path.join(resultfolder, 'test_rpfs_wct_worstcase_time_spent.csv')

### Create the output folder 

In [110]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'tables')
if not os.path.exists(outputfolder_table):
    os.makedirs(outputfolder_table)

### Process consolidated CSV result files

In [132]:
df = pd.read_csv(rpfs_file)
df = df.rename(columns=lambda x: x.strip())
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('""', '').str.replace('""', '')

In [133]:
df

,instance,gamma,fractional_solution,bigm_type,solver,time_spent
0,RB0101009_10_5_10_wct_inputs.txt,30,true,3,"""Gurobi""",18.551797
1,RB0101009_10_5_10_wct_inputs.txt,30,false,2,"""Gurobi""",22.722597
2,RB0101009_10_5_10_wct_inputs.txt,30,true,3,"""CPLEX""",39.938303
3,RB0101009_10_5_10_wct_inputs.txt,30,false,1,"""Gurobi""",44.445573
4,RB0101009_10_5_10_wct_inputs.txt,30,false,3,"""Gurobi""",16.147948
...,...,...,...,...,...,...
1195,RB0103007_10_5_30_wct_inputs.txt,40,true,1,"""CPLEX""",30.286308
1196,RB0103007_10_5_30_wct_inputs.txt,40,false,3,"""CPLEX""",14.951206
1197,RB0103007_10_5_30_wct_inputs.txt,40,false,1,"""CPLEX""",31.708677
1198,RB0103007_10_5_30_wct_inputs.txt,40,true,2,"""Gurobi""",6.089671


In [135]:
df['fractional_solution'] = df['fractional_solution'].str.replace('"', '').str.replace('"', '').str.strip()
d = {'true': True, 'false': False}
df["fractional_solution"] = df["fractional_solution"].map(d)

In [136]:
df['solver'] = df['solver'].str.replace('"', '').str.replace('"', '').str.strip()

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   instance             1200 non-null   object 
 1   gamma                1200 non-null   int64  
 2   fractional_solution  1200 non-null   bool   
 3   bigm_type            1200 non-null   int64  
 4   solver               1200 non-null   object 
 5   time_spent           1200 non-null   float64
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 48.2+ KB


In [138]:
df.head(4)

,instance,gamma,fractional_solution,bigm_type,solver,time_spent
0,RB0101009_10_5_10_wct_inputs.txt,30,True,3,Gurobi,18.551797
1,RB0101009_10_5_10_wct_inputs.txt,30,False,2,Gurobi,22.722597
2,RB0101009_10_5_10_wct_inputs.txt,30,True,3,CPLEX,39.938303
3,RB0101009_10_5_10_wct_inputs.txt,30,False,1,Gurobi,44.445573


# Tables

In [139]:
df_ = pd.pivot_table(df, values=['time_spent'], index=['instance', 'gamma', 'fractional_solution', 'bigm_type'],
                    columns=['solver'], aggfunc=np.sum, fill_value=0)
df_.columns = df_.columns.to_flat_index()
df_

(time_spent, CPLEX)  \
instance                         gamma fractional_solution bigm_type                        
RB0101001_10_5_10_wct_inputs.txt 30    False               1                    48.274218   
                                                           2                    38.571727   
                                                           3                    18.296678   
                                       True                1                    52.161927   
                                                           2                    38.633717   
...                                                                                   ...   
RB0105010_10_5_50_wct_inputs.txt 40    False               2                   143.048404   
                                                           3                    62.894007   
                                       True                1                   108.286586   
                                                           2                    30.736441   
                                                           3                    67.361079   

                                                                      (time_spent, Gurobi)  
instance                         gamma fractional_solution bigm_type                        
RB0101001_10_5_10_wct_inputs.txt 30    False               1                      7.862524  
                                                           2                     16.427593  
                                                           3                     14.683508  
                                       True                1                     12.824416  
                                                           2                     19.325409  
...                                                                                    ...  
RB0105010_10_5_50_wct_inputs.txt 40    False               2                     46.176740  
                                                           3                     19.683683  
                                       True                1                     12.905464  
                                                           2                     32.237307  
                                                           3                     27.795342  

[600 rows x 2 columns]

In [140]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 600 entries, ('RB0101001_10_5_10_wct_inputs.txt', 30, False, 1) to ('RB0105010_10_5_50_wct_inputs.txt', 40, True, 3)
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (time_spent, CPLEX)   600 non-null    float64
 1   (time_spent, Gurobi)  600 non-null    float64
dtypes: float64(2)
memory usage: 12.4+ KB


In [141]:
df_['Gurobi_wins'] = (df_[('time_spent', 'CPLEX')] >= df_[('time_spent', 'Gurobi')]).astype(int)

In [142]:
print(df_['Gurobi_wins'].sum())
print(len(df_.index))

554
600


In [143]:
df_[(df_['Gurobi_wins'] == 0)]

(time_spent, CPLEX)  \
instance                         gamma fractional_solution bigm_type                        
RB0101002_10_5_10_wct_inputs.txt 40    True                1                    19.661148   
                                                           3                    16.533206   
RB0101004_10_5_10_wct_inputs.txt 30    False               3                     7.976628   
RB0101007_10_5_10_wct_inputs.txt 40    False               3                     4.982446   
                                       True                3                     6.583502   
RB0101008_10_5_10_wct_inputs.txt 30    True                2                    14.000326   
                                 40    True                3                    14.291689   
RB0102003_10_5_20_wct_inputs.txt 40    False               3                    23.609039   
RB0102004_10_5_20_wct_inputs.txt 30    False               3                     6.342815   
                                 40    False               3                     6.160861   
RB0102005_10_5_20_wct_inputs.txt 30    True                2                    52.408474   
RB0102007_10_5_20_wct_inputs.txt 40    False               2                    10.310406   
RB0102008_10_5_20_wct_inputs.txt 30    False               3                    47.222267   
                                 40    True                2                    14.246461   
RB0102009_10_5_20_wct_inputs.txt 40    False               1                    35.785494   
RB0102010_10_5_20_wct_inputs.txt 30    False               3                    30.199007   
RB0103001_10_5_30_wct_inputs.txt 40    False               1                    43.939133   
RB0103005_10_5_30_wct_inputs.txt 30    True                3                    38.492100   
RB0103007_10_5_30_wct_inputs.txt 30    False               2                    13.844283   
RB0103008_10_5_30_wct_inputs.txt 40    True                2                    25.072537   
RB0103009_10_5_30_wct_inputs.txt 40    True                3                    11.241067   
RB0103010_10_5_30_wct_inputs.txt 40    True                3                    43.101262   
RB0104002_10_5_40_wct_inputs.txt 40    False               3                    24.225077   
RB0104003_10_5_40_wct_inputs.txt 30    False               3                   213.836222   
RB0104004_10_5_40_wct_inputs.txt 30    True                2                    16.094847   
                                                           3                    18.296427   
RB0104005_10_5_40_wct_inputs.txt 30    False               1                    87.528201   
                                       True                3                    28.702724   
                                 40    False               1                    51.403572   
RB0104007_10_5_40_wct_inputs.txt 30    True                2                    11.659804   
                                 40    True                3                    24.906493   
RB0104008_10_5_40_wct_inputs.txt 40    False               1                    27.879321   
RB0104010_10_5_40_wct_inputs.txt 30    False               2                    57.986933   
                                       True                2                    69.832044   
RB0105001_10_5_50_wct_inputs.txt 30    True                1                    43.213108   
                                 40    False               3                    28.278133   
RB0105003_10_5_50_wct_inputs.txt 40    True                2                   102.855683   
RB0105004_10_5_50_wct_inputs.txt 30    False               2                    15.799538   
                                 40    False               2                    17.770272   
RB0105006_10_5_50_wct_inputs.txt 40    True                1                    60.489666   
RB0105008_10_5_50_wct_inputs.txt 30    False               2                    25.452805   
                                                           3           

### Conclusion: when CPLEX wins, it is a narrow victory. So we can ignore these cases.

In [147]:
df_.reset_index().groupby(by=['Gurobi_wins']).sum().plot.pie(y='fractional_solution', figsize=(5, 5))

KeyError: 'Gurobi_wins'

### Let's now analyze which Gurobi configuration is better

In [148]:
df_gurobi = df[df['solver'] == 'Gurobi']
df_gurobi

,instance,gamma,fractional_solution,bigm_type,solver,time_spent
0,RB0101009_10_5_10_wct_inputs.txt,30,True,3,Gurobi,18.551797
1,RB0101009_10_5_10_wct_inputs.txt,30,False,2,Gurobi,22.722597
3,RB0101009_10_5_10_wct_inputs.txt,30,False,1,Gurobi,44.445573
4,RB0101009_10_5_10_wct_inputs.txt,30,False,3,Gurobi,16.147948
6,RB0101009_10_5_10_wct_inputs.txt,30,True,1,Gurobi,15.127756
...,...,...,...,...,...,...
1189,RB0103007_10_5_30_wct_inputs.txt,40,False,2,Gurobi,6.435572
1191,RB0103007_10_5_30_wct_inputs.txt,40,False,1,Gurobi,11.112334
1192,RB0103007_10_5_30_wct_inputs.txt,40,False,3,Gurobi,7.125073
1194,RB0103007_10_5_30_wct_inputs.txt,40,True,1,Gurobi,8.564471


In [149]:
df_ = pd.pivot_table(df_gurobi, values=['time_spent'], index=['instance', 'gamma', 'fractional_solution'],
                    columns=['bigm_type'], aggfunc=np.sum, fill_value=0)
df_.columns = df_.columns.to_flat_index()
df_

(time_spent, 1)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                       7.862524   
                                       True                       12.824416   
                                 40    False                      15.605349   
                                       True                       16.511851   
RB0101002_10_5_10_wct_inputs.txt 30    False                       7.361534   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       18.300398   
RB0105010_10_5_50_wct_inputs.txt 30    False                      22.184362   
                                       True                      147.971099   
                                 40    False                      38.392248   
                                       True                       12.905464   

                                                            (time_spent, 2)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                      16.427593   
                                       True                       19.325409   
                                 40    False                       9.039581   
                                       True                        9.330266   
RB0101002_10_5_10_wct_inputs.txt 30    False                       6.573118   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       22.215767   
RB0105010_10_5_50_wct_inputs.txt 30    False                      20.447717   
                                       True                       21.825709   
                                 40    False                      46.176740   
                                       True                       32.237307   

                                                            (time_spent, 3)  
instance                         gamma fractional_solution                   
RB0101001_10_5_10_wct_inputs.txt 30    False                      14.683508  
                                       True                       11.703198  
                                 40    False                       8.664781  
                                       True                       12.756748  
RB0101002_10_5_10_wct_inputs.txt 30    False                      10.824456  
...                                                                     ...  
RB0105009_10_5_50_wct_inputs.txt 40    True                       51.607066  
RB0105010_10_5_50_wct_inputs.txt 30    False                      23.331858  
                                       True                       52.481066  
                                 40    False                      19.683683  
                                       True                       27.795342  

[200 rows x 3 columns]

In [150]:
df_.columns

Index([('time_spent', 1), ('time_spent', 2), ('time_spent', 3)], dtype='object')

In [151]:
df_['min_time'] = df_.agg("min", axis="columns")
df_

(time_spent, 1)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                       7.862524   
                                       True                       12.824416   
                                 40    False                      15.605349   
                                       True                       16.511851   
RB0101002_10_5_10_wct_inputs.txt 30    False                       7.361534   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       18.300398   
RB0105010_10_5_50_wct_inputs.txt 30    False                      22.184362   
                                       True                      147.971099   
                                 40    False                      38.392248   
                                       True                       12.905464   

                                                            (time_spent, 2)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                      16.427593   
                                       True                       19.325409   
                                 40    False                       9.039581   
                                       True                        9.330266   
RB0101002_10_5_10_wct_inputs.txt 30    False                       6.573118   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       22.215767   
RB0105010_10_5_50_wct_inputs.txt 30    False                      20.447717   
                                       True                       21.825709   
                                 40    False                      46.176740   
                                       True                       32.237307   

                                                            (time_spent, 3)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                      14.683508   
                                       True                       11.703198   
                                 40    False                       8.664781   
                                       True                       12.756748   
RB0101002_10_5_10_wct_inputs.txt 30    False                      10.824456   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       51.607066   
RB0105010_10_5_50_wct_inputs.txt 30    False                      23.331858   
                                       True                       52.481066   
                                 40    False                      19.683683   
                                       True                       27.795342   

                                                             min_time  
instance                         gamma fractional_solution             
RB0101001_10_5_10_wct_inputs.txt 30    False                 7.862524  
                                       True                 11.703198  
                                 40    False                 8.664781  
                                       True                  9.330266  
RB0101002_10_5_10_wct_inputs.txt 30    False                 6.573118  
...                                                               ...  
RB0105009_10_5_50_wct_inputs.txt 40    True                 18.300398  
RB0105010_10_5_50_wct_inputs.txt 30    False                20.447717  
                                       True                 21.825709  
                                 40    False                19.683683  
                                       True                 12.905464  

[200 rows x 4 columns]

In [152]:
df_['bigM1_wins'] = (df_[('time_spent', 1)] == df_['min_time']).astype(int)
df_['bigM2_wins'] = (df_[('time_spent', 2)] == df_['min_time']).astype(int)
df_['bigM3_wins'] = (df_[('time_spent', 3)] == df_['min_time']).astype(int)
df_

(time_spent, 1)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                       7.862524   
                                       True                       12.824416   
                                 40    False                      15.605349   
                                       True                       16.511851   
RB0101002_10_5_10_wct_inputs.txt 30    False                       7.361534   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       18.300398   
RB0105010_10_5_50_wct_inputs.txt 30    False                      22.184362   
                                       True                      147.971099   
                                 40    False                      38.392248   
                                       True                       12.905464   

                                                            (time_spent, 2)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                      16.427593   
                                       True                       19.325409   
                                 40    False                       9.039581   
                                       True                        9.330266   
RB0101002_10_5_10_wct_inputs.txt 30    False                       6.573118   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       22.215767   
RB0105010_10_5_50_wct_inputs.txt 30    False                      20.447717   
                                       True                       21.825709   
                                 40    False                      46.176740   
                                       True                       32.237307   

                                                            (time_spent, 3)  \
instance                         gamma fractional_solution                    
RB0101001_10_5_10_wct_inputs.txt 30    False                      14.683508   
                                       True                       11.703198   
                                 40    False                       8.664781   
                                       True                       12.756748   
RB0101002_10_5_10_wct_inputs.txt 30    False                      10.824456   
...                                                                     ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                       51.607066   
RB0105010_10_5_50_wct_inputs.txt 30    False                      23.331858   
                                       True                       52.481066   
                                 40    False                      19.683683   
                                       True                       27.795342   

                                                             min_time  \
instance                         gamma fractional_solution              
RB0101001_10_5_10_wct_inputs.txt 30    False                 7.862524   
                                       True                 11.703198   
                                 40    False                 8.664781   
                                       True                  9.330266   
RB0101002_10_5_10_wct_inputs.txt 30    False                 6.573118   
...                                                               ...   
RB0105009_10_5_50_wct_inputs.txt 40    True                 18.300398   
RB0105010_10_5_50_wct_inputs.txt 30    False                20.447717   
                                       True                 21.825709   
                                 40    False                19.683683   
                                       True                 12.905464   

                          

In [153]:
df_t = df_.reset_index()

### For `fractional_solution == true`, the best option is Gurobi solver with `bigM_type == 3`.

In [154]:
df_t[df_t['fractional_solution'] == True].describe()

,gamma,"(time_spent, 1)","(time_spent, 2)","(time_spent, 3)",min_time,bigM1_wins,bigM2_wins,bigM3_wins
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,35.000000,24.957742,26.722800,23.114489,13.691719,0.310000,0.330000,0.360000
std,5.025189,24.182789,25.941979,17.888200,9.269175,0.464823,0.472582,0.482418
min,30.000000,4.971632,4.185057,1.094489,1.094489,0.000000,0.000000,0.000000
25%,30.000000,12.120068,10.618747,8.868210,8.186797,0.000000,0.000000,0.000000
50%,35.000000,15.654391,15.523270,16.671605,11.181036,0.000000,0.000000,0.000000
75%,40.000000,28.278788,32.303212,31.812349,15.022430,1.000000,1.000000,1.000000
max,40.000000,147.971099,128.100864,76.002428,54.127722,1.000000,1.000000,1.000000


In [155]:
df_t[df_t['fractional_solution'] == True].sum()

instance               RB0101001_10_5_10_wct_inputs.txtRB0101001_10_5...
gamma                                                               3500
fractional_solution                                                  100
(time_spent, 1)                                                  2495.77
(time_spent, 2)                                                  2672.28
(time_spent, 3)                                                  2311.45
min_time                                                         1369.17
bigM1_wins                                                            31
bigM2_wins                                                            33
bigM3_wins                                                            36
dtype: object

### For `fractional_solution == false`, the best option is Gurobi solver with `bigM_type == 2`.

In [156]:
df_t[df_t['fractional_solution'] == False].describe()

,gamma,"(time_spent, 1)","(time_spent, 2)","(time_spent, 3)",min_time,bigM1_wins,bigM2_wins,bigM3_wins
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,35.000000,26.830104,23.993141,31.840675,13.981666,0.250000,0.410000,0.340000
std,5.025189,22.478457,22.785684,45.287541,8.964384,0.435194,0.494311,0.476095
min,30.000000,2.665886,2.311682,3.670282,2.311682,0.000000,0.000000,0.000000
25%,30.000000,11.927266,9.032432,10.321229,8.072692,0.000000,0.000000,0.000000
50%,35.000000,18.341103,15.924063,14.994523,11.804389,0.000000,0.000000,0.000000
75%,40.000000,36.866316,29.010548,32.838168,16.083756,0.250000,1.000000,1.000000
max,40.000000,128.530537,120.128255,324.606401,58.444741,1.000000,1.000000,1.000000


In [157]:
df_t[df_t['fractional_solution'] == False].sum()

instance               RB0101001_10_5_10_wct_inputs.txtRB0101001_10_5...
gamma                                                               3500
fractional_solution                                                    0
(time_spent, 1)                                                  2683.01
(time_spent, 2)                                                  2399.31
(time_spent, 3)                                                  3184.07
min_time                                                         1398.17
bigM1_wins                                                            25
bigM2_wins                                                            41
bigM3_wins                                                            34
dtype: object